In [5]:
from pyspark.sql import SparkSession
spark=SparkSession

In [6]:
spark=SparkSession.builder.appName('Prediction').getOrCreate()

In [7]:
df_spark=spark.read.csv('MOCK_DATAPrediction.csv',header=True,inferSchema=True)

In [8]:
df_spark.show()

+---------+------+----------+---+
|     name|Salary|YearsOfExp|Age|
+---------+------+----------+---+
|  Ginelle|  4207|         1|  1|
|  Jordana|  8160|         0|  0|
|Corabelle|  1158|         7|  7|
|   Freddy|  2656|         3|  3|
|  Glendon|  3398|         2|  3|
|  Vernice|  4463|         6|  3|
|  Jacquie|  2809|         0|  2|
|   Randal|  6060|         2|  1|
|    Uriel|  1069|         7|  6|
|    Fanni|  4496|         4|  7|
|    Lindy|  2701|         6|  9|
|   Leonie|  1164|         3|  1|
|    Irwin|  4742|         3|  0|
|   Jenine|   195|         9|  1|
|  Osbourn|  4314|         8|  9|
|   Stormy|  4698|         1|  4|
|      Zoe|  2844|         5|  3|
|    Gabey|  8677|         5|  7|
|     Cher|  1661|         0|  9|
|   Darcee|  8864|         0|  2|
+---------+------+----------+---+
only showing top 20 rows



In [9]:
df_spark.printSchema()

root
 |-- name: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- YearsOfExp: integer (nullable = true)
 |-- Age: integer (nullable = true)



In [10]:
df_spark.columns

['name', 'Salary', 'YearsOfExp', 'Age']

In [11]:
[YearsOfExp,Age]----> new feature---->independent feature

In [15]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["YearsOfExp","Age"],outputCol="Independent Features")

In [16]:
output=featureassembler.transform(df_spark)

In [17]:
output.show()

+---------+------+----------+---+--------------------+
|     name|Salary|YearsOfExp|Age|Independent Features|
+---------+------+----------+---+--------------------+
|  Ginelle|  4207|         1|  1|           [1.0,1.0]|
|  Jordana|  8160|         0|  0|           (2,[],[])|
|Corabelle|  1158|         7|  7|           [7.0,7.0]|
|   Freddy|  2656|         3|  3|           [3.0,3.0]|
|  Glendon|  3398|         2|  3|           [2.0,3.0]|
|  Vernice|  4463|         6|  3|           [6.0,3.0]|
|  Jacquie|  2809|         0|  2|           [0.0,2.0]|
|   Randal|  6060|         2|  1|           [2.0,1.0]|
|    Uriel|  1069|         7|  6|           [7.0,6.0]|
|    Fanni|  4496|         4|  7|           [4.0,7.0]|
|    Lindy|  2701|         6|  9|           [6.0,9.0]|
|   Leonie|  1164|         3|  1|           [3.0,1.0]|
|    Irwin|  4742|         3|  0|           [3.0,0.0]|
|   Jenine|   195|         9|  1|           [9.0,1.0]|
|  Osbourn|  4314|         8|  9|           [8.0,9.0]|
|   Stormy

In [18]:
output.columns

['name', 'Salary', 'YearsOfExp', 'Age', 'Independent Features']

In [19]:
final_data=output.select("Independent Features","Salary")

In [20]:
final_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|           [1.0,1.0]|  4207|
|           (2,[],[])|  8160|
|           [7.0,7.0]|  1158|
|           [3.0,3.0]|  2656|
|           [2.0,3.0]|  3398|
|           [6.0,3.0]|  4463|
|           [0.0,2.0]|  2809|
|           [2.0,1.0]|  6060|
|           [7.0,6.0]|  1069|
|           [4.0,7.0]|  4496|
|           [6.0,9.0]|  2701|
|           [3.0,1.0]|  1164|
|           [3.0,0.0]|  4742|
|           [9.0,1.0]|   195|
|           [8.0,9.0]|  4314|
|           [1.0,4.0]|  4698|
|           [5.0,3.0]|  2844|
|           [5.0,7.0]|  8677|
|           [0.0,9.0]|  1661|
|           [0.0,2.0]|  8864|
+--------------------+------+
only showing top 20 rows



In [21]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=final_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor=regressor.fit(train_data)

In [22]:
### Coeffs
regressor.coefficients

DenseVector([-52.2075, 12.3434])

In [23]:
#Intercepts
regressor.intercept

4947.017236823101

In [24]:
pred_results=regressor.evaluate(test_data)

In [25]:
pred_results.predictions.show()

/usr/local/anaconda3/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|           (2,[],[])|  1483| 4947.017236823101|
|           (2,[],[])|  4146| 4947.017236823101|
|           [0.0,1.0]|  2002| 4959.360612759985|
|           [0.0,1.0]|  3036| 4959.360612759985|
|           [0.0,1.0]|  3768| 4959.360612759985|
|           [0.0,1.0]|  9817| 4959.360612759985|
|           [0.0,2.0]|   290|  4971.70398869687|
|           [0.0,2.0]|  1504|  4971.70398869687|
|           [0.0,2.0]|  3783|  4971.70398869687|
|           [0.0,2.0]|  5289|  4971.70398869687|
|           [0.0,2.0]|  5459|  4971.70398869687|
|           [0.0,3.0]|  6372| 4984.047364633755|
|           [0.0,3.0]|  6566| 4984.047364633755|
|           [0.0,4.0]|  5879|4996.3907405706395|
|           [0.0,5.0]|  5237| 5008.734116507524|
|           [0.0,5.0]|  9083| 5008.734116507524|
|           [0.0,5.0]|  9418| 5008.734116507524|
|           [0.0,6.0

In [26]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(2435.728477946055, 7950627.083068863)